In [2]:
import neurokit2 as nk
import pandas as pd
import numpy as np
from preprocessing import *

In [154]:
df = open_brux_csv(1, 1, '1022102')
print(f"dfshape: {df.shape}")
ecg = df["ECG"]

MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type int64

In [133]:
ecg

0           0.234100
1           0.234080
2           0.234100
3           0.234110
4           0.234090
              ...   
26401132    0.013077
26401133    0.013457
26401134    0.013788
26401135    0.013927
26401136         NaN
Name: ECG, Length: 26401137, dtype: float64

In [112]:
def peaks_to_rri(peaks, sampling_rate=2000, interpolate=False, **kwargs):
    rri = np.diff(peaks) / sampling_rate * 1000

    if interpolate is False:
        return rri

    else:

        # Minimum sampling rate for interpolation
        if sampling_rate < 10:
            sampling_rate = 10

        # Compute length of interpolated heart period signal at requested sampling rate.
        desired_length = int(np.rint(peaks[-1]))

        rri = nk.signal_interpolate(
            peaks[1:],  # Skip first peak since it has no corresponding element in heart_period
            rri,
            x_new=np.arange(desired_length),
            **kwargs
        )
        return rri, sampling_rate

In [20]:

def compute_HRV(rpeaks_clean, sampling_rate=2000, window=5):    
    # Regular HRV matrix (from peaks)
    duration_peaks=rpeaks_clean[len(rpeaks_clean)-1] #gives me the duration in samples
    divider=duration_peaks/sampling_rate/60/window  #sampling_rate, 5 min window segments
    segment=np.array_split(rpeaks_clean,divider) #divide in segments of 5 min; the last segment may be shorter; discard during statistical analysis on HRV metrics

    hrv_final_df=pd.DataFrame()

    for i in range(len(segment)):
        hrv_segment = nk.hrv(segment[i], sampling_rate=sampling_rate, show=False)
        print(i)
        hrv_final_df = pd.concat([hrv_segment_df, hrv_segment],ignore_index=True)


    return hrv_final_df

In [134]:

# Filter the data with ranges specified from Barbara
filter_ecg = nk.signal_filter(ecg, sampling_rate=2000, lowcut=0.5, highcut=150)
# clean ecg signal
ecg_cleaned = nk.ecg_clean(filter_ecg, sampling_rate=2000, method="pantompkins1985")
# find r peaks
rpeaks, info = nk.ecg_peaks(ecg_cleaned, sampling_rate=2000)

c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\neurokit2\ecg\ecg_clean.py:93: NeuroKitWarning: There are 1 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\neurokit2\ecg\ecg_clean.py:142: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  ecg_signal = pd.DataFrame.pad(pd.Series(ecg_signal))


RangeIndex(start=0, stop=26401137, step=1)

In [135]:
# Trim trailing zeros
#rpeaks = np.trim_zeros(rpeaks,trim='b')
# Artifact removal
# nk.signal_fixpeaks saves the corrected peak locations to the [1] index of the output data sturcture
# accessible like so: clean_peaks[1]
rpeaks_clean=nk.signal_fixpeaks(rpeaks, sampling_rate=2000, iterative=False, show=False,interval_min=0.33,interval_max=0.75, method="kubios")
# Convert peaks to rri - not useful for our case
rri = peaks_to_rri(rpeaks_clean[1], sampling_rate=2000, interpolate=False)



In [136]:
rpeaks_clean

({'ectopic': [151,
   628,
   675,
   741,
   788,
   807,
   867,
   964,
   994,
   1107,
   1208,
   1212,
   1213,
   1227,
   1480,
   1851,
   1912,
   2061,
   2135,
   2238,
   2330,
   2388,
   2418,
   2482,
   2568,
   2900,
   3180,
   3203,
   3212,
   3855,
   3930,
   4152,
   4389,
   4460,
   4585,
   4767,
   4768,
   5089,
   5928,
   6401,
   6983,
   7054,
   7264,
   7326,
   7493,
   7573,
   7641,
   8137,
   8417,
   8555,
   8775,
   9000,
   9294,
   9543,
   9797,
   12346,
   12347,
   12464,
   12465,
   12554,
   12638],
  'missed': [],
  'extra': [2038,
   3074,
   4811,
   5376,
   7342,
   8527,
   9154,
   9167,
   9171,
   9274,
   9837,
   11772,
   12055,
   12863,
   13328],
  'longshort': [171,
   222,
   223,
   269,
   441,
   537,
   560,
   561,
   1141,
   1320,
   1321,
   1389,
   1457,
   1458,
   1799,
   1800,
   1904,
   2039,
   2060,
   2481,
   2801,
   2802,
   3075,
   3725,
   3866,
   3929,
   3931,
   3932,
   4433,
   4766,
  

In [139]:
rpeaks_clean[1][151]

246242

In [126]:
rpeaks_clean = rpeaks_clean[1]

In [127]:
# Compute HRV metrics

duration_peaks=rpeaks_clean[len(rpeaks_clean)-1] #gives me the duration in samples
divider=duration_peaks/2000/60/5  #sampling_rate, 5 min window segments
segment=np.array_split(rpeaks_clean,divider) #divide in segments of 5 min; the last segment may be shorter; discard during statistical analysis on HRV metrics

hrv_segment_df=pd.DataFrame()

for i in range(len(segment)):
    hrv_segment = nk.hrv(segment[i], sampling_rate=2000, show=False)
    print(i)
    hrv_segment_df = pd.concat([hrv_segment_df, hrv_segment],ignore_index=True)

0
1
2
3
4


c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


5


c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


6
7
8
9
10
11


...

32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


49
50
51
52
53
54
55


c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [128]:
hrv_segment_df

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,810.521236,340.495820,58.494089,338.047195,NaN,NaN,NaN,NaN,410.279570,411.074770,...,0.585036,7.747409,0.882763,0.755277,1.119778,1.147820,1.058777,1.900703,2.212144,0.990494
1,795.249035,163.759717,24.864825,163.897725,NaN,NaN,NaN,NaN,149.385330,149.668783,...,0.907922,7.728179,0.775825,1.144755,1.211037,1.718931,1.424064,1.775881,2.499694,0.711917
2,917.471042,95.773636,21.372337,78.781048,NaN,NaN,NaN,NaN,82.092540,82.252093,...,1.128220,7.351543,0.968141,0.705329,1.232897,1.385512,1.469575,1.836239,2.468887,0.959541
3,853.687259,139.152302,78.189891,104.854383,NaN,NaN,NaN,NaN,108.098971,108.307294,...,0.931800,7.639301,0.823898,0.934205,1.211133,1.676725,1.446716,1.749639,2.431097,0.588106
4,937.187259,70.703282,14.521822,69.209482,NaN,NaN,NaN,NaN,75.275997,75.418866,...,1.669542,7.462180,1.324887,1.222744,1.357903,1.862498,1.782729,1.882510,3.161351,0.990494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1006.686047,105.296433,42.336180,91.432958,NaN,NaN,NaN,NaN,56.551808,56.657209,...,1.368459,7.696172,0.940403,1.538618,1.403005,1.910033,1.756934,1.585460,2.916717,0.683128
113,922.000000,122.532938,57.463707,107.406493,NaN,NaN,NaN,NaN,70.642280,70.778029,...,0.932320,7.621579,0.700174,1.136294,1.166199,1.426894,1.516489,1.566875,1.923795,0.527872
114,927.889535,120.565788,29.016084,110.166915,NaN,NaN,NaN,NaN,65.334972,65.459648,...,1.090307,7.668090,0.755871,1.220708,1.275161,1.866790,1.617971,1.584870,2.124631,0.652077
115,890.885659,177.888675,136.546707,128.104865,NaN,NaN,NaN,NaN,63.422415,63.545993,...,0.762046,7.757313,0.590861,1.278952,1.164545,1.771057,1.494798,1.403062,2.257164,0.527872


In [129]:
def get_herzig_ranges():
    return {
        'nrem': {'min': 0.31, 'median': (2.02-0.31)/2 , 'max': 2.02},
        'rem' : {'min': 1.30, 'median': 2.02 , 'max': 3.22}
    }

In [130]:
window_size = 5*60*2000
start = 0
end = window_size
x=0
y=0
values = []
for i, row in hrv_segment_df.iterrows():
    print(row['HRV_LFHF'])
    nrem = get_herzig_ranges()['nrem']
    rem = get_herzig_ranges()['rem']


    if  nrem['min'] <= row['HRV_LFHF'] <= nrem['max']:
        stage = 'nrem'

    if row['HRV_LFHF'] < nrem['min']:
        stage = 'nrem'

    if rem['min'] <= row['HRV_LFHF'] <= rem['max']:
        stage = 'rem'

    if row['HRV_LFHF'] > rem['max']:
        stage = 'rem'

    values.append({
            'start_id': start,
            'end_id': end,
            'LF_HF': row['HRV_LFHF'],
            'SD': row['HRV_SDNN'],
            'stage': stage,
            'y': y,
            'x': x
        })

    if x==((90/5)-1):
        x=0
        y+=1

    else:
        x+=1

        

    start = end
    end += window_size


1.6648325991997475
0.9418067521951843
0.9079243265017617
1.4105817216609071
0.7507939155480371
164.3739958281372
84.61669226754789
32.13058786986925
2.0046055678622348
1.7988535862575679
0.5849745797441904
1.9077986892625887
630.2399244395511
1.4555419638938645
2.9926373915439983
2.22859772291126
5.8301002463885325
1.08320674283628
0.36436866281280655
0.3984839002584155
0.6214587458772611
1.122115315557529
0.3064439651872423
4.375102354543225
0.6640790160324368
0.9096736411712139
1.4221343906949182
1.008776181708114
0.6157259318909784
0.6680517683285865
3.8711818183894446
137.98566739583256
50.2938863573499
8.739878860316685
3.4091852197254635
0.6222432601821053
1.7518064085794223
0.5086263910361754
0.5493285193573085
0.9978930366979267
75.94796109812347
0.6724964514342976
2.0286611767185883
0.5762331995628961
1.6736495337485195
0.5253579863705863
1.1057664800635163
2.62482948391512
87.06762281402851
93.12374568998993
4.501380775545347
2.2961279508859875
4.654770268404622
2.49774221425

In [131]:
values

[{'start_id': 0,
  'end_id': 600000,
  'LF_HF': 1.6648325991997475,
  'SD': 340.4958202637328,
  'stage': 'rem',
  'y': 0,
  'x': 0},
 {'start_id': 600000,
  'end_id': 1200000,
  'LF_HF': 0.9418067521951843,
  'SD': 163.7597169998024,
  'stage': 'nrem',
  'y': 0,
  'x': 1},
 {'start_id': 1200000,
  'end_id': 1800000,
  'LF_HF': 0.9079243265017617,
  'SD': 95.773635591902,
  'stage': 'nrem',
  'y': 0,
  'x': 2},
 {'start_id': 1800000,
  'end_id': 2400000,
  'LF_HF': 1.4105817216609071,
  'SD': 139.1523016485318,
  'stage': 'rem',
  'y': 0,
  'x': 3},
 {'start_id': 2400000,
  'end_id': 3000000,
  'LF_HF': 0.7507939155480371,
  'SD': 70.70328238034853,
  'stage': 'nrem',
  'y': 0,
  'x': 4},
 {'start_id': 3000000,
  'end_id': 3600000,
  'LF_HF': 164.3739958281372,
  'SD': 16064.716973036286,
  'stage': 'rem',
  'y': 0,
  'x': 5},
 {'start_id': 3600000,
  'end_id': 4200000,
  'LF_HF': 84.61669226754789,
  'SD': 18272.26202950274,
  'stage': 'rem',
  'y': 0,
  'x': 6},
 {'start_id': 4200000